<a href="https://colab.research.google.com/github/smaranjitghose/quantum_burglary/blob/master/quantum_burglary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting our dependecies

In [83]:
!pip install qiskit

In [15]:
#!pip install pycryptodomex --no-binary :all:
!pip install rsa

  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
Using cached rsa-4.9.1-py3-none-any.whl (34 kB)



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import numpy as np
from qiskit import *
from math import sqrt,log,gcd
import random
from random import randint
import rsa
#from Crypto.PublicKey import RSA

# Lets make our RSA Algorithm

![flow-chart](https://i.ytimg.com/vi/-jSX9fNJiN8/maxresdefault.jpg)

#### Calculating Modular  Inverse

In [19]:
def mod_inverse(a, m):
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return -1

#### Checking for primality

In [21]:
def isprime(n):
    if n < 2:
        return False
    elif n == 2:
        return True
    else:
        for i in range(1, int(sqrt(n)) + 1):
            if n % i == 0:
                return False
    return True

## Generating Key Value Pairs

In [23]:
def generate_keypair(keysize):
    p = randint(1, 1000)
    q = randint(1, 1000)
    nMin = 1 << (keysize - 1)
    nMax = (1 << keysize) - 1
    primes = [2]
    start = 1 << (keysize // 2 - 1)
    stop = 1 << (keysize // 2 + 1)
    if start >= stop:
        return []
    for i in range(3, stop + 1, 2):
        for p in primes:
            if i % p == 0:
                break
        else:
            primes.append(i)
    while (primes and primes[0] < start):
        del primes[0]
    # Select two random prime numbers p and q
    while primes:
        p = random.choice(primes)
        primes.remove(p)
        q_values = [q for q in primes if nMin <= p * q <= nMax]
        if q_values:
            q = random.choice(q_values)
            break
    # Calculate n
    n = p * q
    # Calculate phi
    phi = (p - 1) * (q - 1)
    # Select e
    e = random.randrange(1, phi)
    g = gcd(e, phi)
    # Calculate d
    while True:
        e = random.randrange(1, phi)
        g = gcd(e, phi)
        d = mod_inverse(e, phi)
        if g == 1 and e != d:
            break

    return ((e, n), (d, n))

## Encryption Step

In [25]:
def encrypt(plaintext, package):
    e, n = package
    ciphertext = [pow(ord(c), e, n) for c in plaintext]
    return ''.join(map(lambda x: str(x), ciphertext)), ciphertext

## Decryption Step

In [27]:
def decrypt(ciphertext, package):
    d, n = package
    plaintext = [chr(pow(c, d, n)) for c in ciphertext]
    return (''.join(plaintext))

# Now lets test with a sample message

#### Generate Keys

In [56]:
!python.exe -m pip install --upgrade pip
!pip install RSA_module

  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 25.1.1
    Uninstalling pip-25.1.1:
      Successfully uninstalled pip-25.1.1


ERROR: Could not find a version that satisfies the requirement RSA_module (from versions: none)
ERROR: No matching distribution found for RSA_module


In [65]:
#import RSA_module

bit_length = int(input("Enter bit length: "))

#public_k, private_k = generate_keypair(2**bit_length)
(public_k, private_k) = rsa.newkeys(1024)


Enter bit length:  2


#### Encryption

In [75]:
plain_txt = input("Enter a message: ")
#cipher_txt, cipher_obj = encrypt(plain_txt, public_k)
cipher_txt = rsa.encrypt(plain_txt.encode(),public_k)
print("Encrypted message: {}".format(cipher_txt))

Enter a message:  tp6tjp6mp6


Encrypted message: b'*\\\xdd\xb862rX\xf1\xb1\x05\xd4:9\x02vK\xdd\x1a\xf7wM\xa5sO\xaca\x8d\x15R{\x91\xd0Aud\xc3\xb5\xe4\xdc\xe6\xfd\r\xa6\\*_P\xc5\x1a0k\xd3N\x18z5.}s\x95\x8cihZ\xbf\xf7\x91?KD\x99\xa1#\x9e\xe1\xa4\xb4\xedRO\xb4\xbb\xfb\x17~\xfd\xddP\xcd\xba\xf1\x1b\x01\x91W\xdc\x08\x8b\x01\x03\xc0i._[\x05pQ\xcd\xd0\n\x93\xb11n\xb4\x911\rd\x9ak\xbc\x81\xfe\xe1^'


#### Decryption

In [79]:
#print("Decrypted message: {}".format(decrypt(cipher_obj, private_k)))
print("Decrypted message: {}".format(rsa.decrypt(cipher_txt, private_k)))

Decrypted message: b'tp6tjp6mp6'


## Now lets frame Shor's Algorithm

In [101]:
from qiskit_aer import AerSimulator as Aer

#qasm_sim = qiskit.Aer.get_backend('qasm_simulator')
#qasm_sim = Aer.get_backend('qasm_simulator')
def period(a,N):
    
    available_qubits = 16 
    r=-1   
    
    if N >= 2**available_qubits:
        print(str(N)+' is too big for IBMQX')
    
    qr = QuantumRegister(available_qubits)   
    cr = ClassicalRegister(available_qubits)
    qc = QuantumCircuit(qr,cr)
    x0 = randint(1, N-1) 
    x_binary = np.zeros(available_qubits, dtype=bool)
    
    for i in range(1, available_qubits + 1):
        bit_state = (N%(2**i)!=0)
        if bit_state:
            N -= 2**(i-1)
        x_binary[available_qubits-i] = bit_state    
    
    for i in range(0,available_qubits):
        if x_binary[available_qubits-i-1]:
            qc.x(qr[i])
    x = x0
    
    while np.logical_or(x != x0, r <= 0):
        r+=1
        qc.measure(qr, cr) 
        for i in range(0,3): 
            qc.x(qr[i])
        qc.cx(qr[2],qr[1])
        qc.cx(qr[1],qr[2])
        qc.cx(qr[2],qr[1])
        qc.cx(qr[1],qr[0])
        qc.cx(qr[0],qr[1])
        qc.cx(qr[1],qr[0])
        qc.cx(qr[3],qr[0])
        qc.cx(qr[0],qr[1])
        qc.cx(qr[1],qr[0])
        
        result = execute(qc,backend = qasm_sim, shots=1024).result()
        counts = result.get_counts()
        
        results = [[],[]]
        for key,value in counts.items(): 
            results[0].append(key)
            results[1].append(int(value))
        s = results[0][np.argmax(np.array(results[1]))]
    return r

In [103]:
def shors_breaker(N):
    N = int(N)
    while True:
        a=randint(0,N-1)
        g=gcd(a,N)
        if g!=1 or N==1:
            return g,N//g
        else:
            r=period(a,N) 
            if r % 2 != 0:
                continue
            elif pow(a,r//2,N)==-1:
                continue
            else:
                p=gcd(pow(a,r//2)+1,N)
                q=gcd(pow(a,r//2)-1,N)
                if p==N or q==N:
                    continue
                return p,q

In [105]:
def modular_inverse(a,m): 
    a = a % m; 
    for x in range(1, m) : 
        if ((a * x) % m == 1) : 
            return x 
    return 1

In [195]:
akey = list(str(public_k))
del akey[0:10]
akey.pop()
akey

['1',
 '0',
 '7',
 '9',
 '3',
 '8',
 '8',
 '2',
 '9',
 '4',
 '9',
 '6',
 '7',
 '1',
 '4',
 '4',
 '2',
 '4',
 '7',
 '3',
 '1',
 '8',
 '6',
 '8',
 '5',
 '5',
 '4',
 '0',
 '5',
 '8',
 '9',
 '6',
 '3',
 '5',
 '2',
 '1',
 '4',
 '6',
 '3',
 '1',
 '6',
 '8',
 '3',
 '5',
 '8',
 '0',
 '2',
 '4',
 '5',
 '5',
 '9',
 '2',
 '1',
 '4',
 '0',
 '6',
 '6',
 '5',
 '9',
 '8',
 '8',
 '4',
 '1',
 '1',
 '3',
 '3',
 '4',
 '7',
 '6',
 '0',
 '1',
 '8',
 '9',
 '5',
 '5',
 '4',
 '0',
 '1',
 '8',
 '1',
 '0',
 '2',
 '0',
 '1',
 '1',
 '1',
 '2',
 '0',
 '2',
 '7',
 '3',
 '7',
 '8',
 '7',
 '3',
 '2',
 '6',
 '0',
 '1',
 '0',
 '6',
 '0',
 '0',
 '3',
 '4',
 '9',
 '0',
 '5',
 '2',
 '2',
 '5',
 '0',
 '7',
 '4',
 '2',
 '7',
 '7',
 '6',
 '5',
 '9',
 '9',
 '4',
 '0',
 '3',
 '8',
 '1',
 '8',
 '5',
 '9',
 '2',
 '4',
 '9',
 '1',
 '8',
 '7',
 '4',
 '1',
 '5',
 '1',
 '0',
 '8',
 '6',
 '7',
 '6',
 '4',
 '6',
 '3',
 '5',
 '4',
 '3',
 '5',
 '2',
 '4',
 '8',
 '6',
 '5',
 '3',
 '9',
 '6',
 '8',
 '7',
 '4',
 '6',
 '6',
 '4',
 '0',
 '0'

只差LIST轉BASE10

In [205]:
#N_shor = public_k[1]
public_k = str(akey)
N_shor = public_k
#assert N_shor > 0,"Input must be positive"
p,q = shors_breaker(N_shor)
phi = (p-1) * (q-1)  
d_shor = modular_inverse(public_k[0], phi) 

ValueError: invalid literal for int() with base 10: "['1', '0', '7', '9', '3', '8', '8', '2', '9', '4', '9', '6', '7', '1', '4', '4', '2', '4', '7', '3', '1', '8', '6', '8', '5', '5', '4', '0', '5', '8', '9', '6', '3', '5', '2', '1', '4', '6', '3', '1'

# Lets Crack our Cipher Text using Shor's Algorithm

In [ ]:
print('Message Cracked using Shors Algorithm: {} '.format(decrypt(cipher_obj, (d_shor,N_shor))))